<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_006_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Ajuste de hiperparámetros con búsqueda aleatoria (Randomized Search)</font>**

<p align="justify">
♥ En el Colab anterior, mostramos cómo usar un enfoque de grid-search para buscar los mejores hiperparámetros que maximicen el rendimiento de generalización de un modelo predictivo.
<br><br>
Sin embargo, un enfoque de grid-search tiene limitaciones. No se escala cuando aumenta el número de parámetros a ajustar.
<br><br>
Además, la grid-search impondrá una regularidad durante la búsqueda que podría ser problemática.
En este Colab, presentaremos otro método para ajustar hiperparámetros denominado randomized search (búsqueda aleatoria) .

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px

 ## **<font color="DeepPink">Carga del conjunto de datos</font>**

In [3]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")
adult_census.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


 ## **<font color="DeepPink">Separamos la variable objetivo y las variables explicativas</font>**

<p align="justify">
👀 Asignamos a un objeto la variable objetivo:
</p>


In [4]:
target_name = "class"
y = adult_census[target_name]
y

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

<p align="justify">
👀 Eliminamos de nuestros datos la variable objetivo y la columna <code>education-num</code> que duplica la información de la columna <code>education</code>.

In [5]:
X = adult_census.drop(columns=[target_name, "education-num"])
X.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


 ## **<font color="DeepPink">Conjunto de entrenamiento y conjunto de prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 ## **<font color="DeepPink">Pipeline</font>**

<p align="justify">
✅ Definiremos un Pipeline que va a manejar características categóricas.

In [8]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

<p align="justify">
✅ Seleccionamos las columnas categoricas.

In [9]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

<p align="justify">
✅ Ahora codificamos con <code>OrdinalEncoder</code>.

In [10]:
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

<p align="justify">
✅ Luego usamos un <code>ColumnTransformer</code> para seleccionar las columnas categóricas y les aplicamos el <code>OrdinalEncoder</code>

In [11]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough')

In [12]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_preprocessor',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

<p align="justify">
✅ Finalmente, usamos un clasificador basado en árboles (histogram gradient-boosting) para predecir si una persona gana o no más de 50 k$ al año.

In [13]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [14]:
model = Pipeline([("preprocessor", preprocessor),
                  ("classifier", HistGradientBoostingClassifier(random_state=42))])

In [15]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(random_state=42))])

 # **<font color="DeepPink">Ajuste mediante randomized search</font>**

<p align="justify">
Con <code>GridSearchCV</code>, los parámetros deben especificarse explícitamente. Ya mencionamos que explorar una gran cantidad de valores para diferentes parámetros será rápidamente intratable.
<br><br>
En su lugar, se propone emplear un enfoque denominado búsqueda aleatoria (randomized search) en contraposición a la búsqueda en grilla (grid search). Esta alternativa consiste en la generación aleatoria de parámetros para la exploración del espacio de hiperparámetros. Un ejemplo visual que ilustra la relevancia limitada de ciertos hiperparámetros, como el hiperparámetro $2$, se presenta en la siguiente figura.

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Optimizacion-001.png?raw=true" width="450">
</p>


<p align="justify">
La búsqueda aleatoria muestra el hiperparámetro $1$ independientemente del hiperparámetro $2$ y encuentra la región óptima.
<br><br>
La clase <code>RandomizedSearchCV</code> permite dicha búsqueda aleatoria. Se usa de manera similar a <code>GridSearchCV</code>, pero es necesario especificar las <b>distribuciones de probabilidad</b> en lugar de los valores de los hiperparámetros. Utilizaremos las funciones <code>uniform</code> y <code>randint</code> de la biblioteca <code>scipy</code>.
<br><br>
La búsqueda aleatoria (con <code>RandomizedSearchCV</code>) suele ser beneficiosa en comparación con la búsqueda en cuadrícula (con <code>GridSearchCV</code>) para optimizar $3$ o más hiperparámetros.

<p align="justify">
Optimizaremos otros $2$ hiperparámetros además de los que optimizamos en el Colab que muestra <code>GridSearchCV</code>:

- `l2_regularización`: corresponde a la regularización;
- `min_samples_leaf`: corresponde al número mínimo de muestras requeridas en un nodo hoja;

<p align="justify">
Los $2$ hiperparámetros ya utilizados son:

- `learning_rate`
- `max_leaf_nodes`


<p align="justify">
Ahora, podemos definir la búsqueda aleatoria usando las diferentes distribuciones.

[Distribución uniforme continua](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

[Distribución uniforme discreta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.randint.html)


In [16]:
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

In [17]:
param_distributions = {'classifier__l2_regularization': uniform(0, 1),
                       'classifier__learning_rate': uniform(0.001, 1),
                       'classifier__max_leaf_nodes': randint(2, 256),
                       'classifier__min_samples_leaf': randint(1, 100)}

In [18]:
model_random_search = RandomizedSearchCV(model,
                                         param_distributions=param_distributions,
                                         n_iter=20,
                                         n_jobs=2,
                                         random_state=123)

`n_iter` para controlar el número de combinaciones aleatorias a evaluar en lugar de definir una grilla completa de hiperparámetros.

In [19]:
model_random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('cat_preprocessor',
                                                                               OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                              unknown_value=-1),
                                                                               ['workclass',
                                                                                'education',
                                                                                'marital-status',
                                                                                'occupation',
                                                                                'relationship',
                                                                                'race',
                                                                                'sex',
                                                                                'native-country'])])),
                                             ('classifier',
                                              HistGradientBoostingClassi...
                   param_distributions={'classifier__l2_regularization': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a933d581930>,
                                        'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7a933d582080>,
                                        'classifier__max_leaf_nodes': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a933d583250>,
                                        'classifier__min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7a933d580d30>},
                   random_state=123)

<p align="justify">
👀 Podemos computar el <code>Accuracy</code> score sobre el conjunto de prueba.

In [20]:
accuracy = model_random_search.score(X_test, y_test)

In [21]:
print("")
print(f"La accuracy del mejor modelo es "
      f"{accuracy:.2f}")


La accuracy del mejor modelo es 0.88


In [22]:
from pprint import pprint

In [23]:
print("")
print("Los mejores parámetros son:")
pprint(model_random_search.best_params_)


Los mejores parámetros son:
{'classifier__l2_regularization': 0.3041207890271841,
 'classifier__learning_rate': 0.4180222110247016,
 'classifier__max_leaf_nodes': 5,
 'classifier__min_samples_leaf': 68}


<p align="justify">
👀 Podemos inspeccionar el resultado, usando el atributo <code>cv_results</code>.

In [24]:
cv_results = pd.DataFrame(model_random_search.cv_results_).sort_values(by='rank_test_score')
cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__l2_regularization,param_classifier__learning_rate,param_classifier__max_leaf_nodes,param_classifier__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
18,1.202598,0.349316,0.126631,0.023520,0.304121,0.418022,5,68,{'classifier__l2_regularization': 0.3041207890...,0.866658,0.869915,0.869506,0.868277,0.867731,0.868418,0.001185,1
4,3.152249,0.278770,0.267544,0.095112,0.438572,0.060678,98,81,{'classifier__l2_regularization': 0.4385722446...,0.865156,0.868550,0.871690,0.865001,0.869779,0.868035,0.002614,2
8,1.170255,0.075158,0.113030,0.005874,0.722443,0.323959,120,95,{'classifier__l2_regularization': 0.7224433825...,0.863109,0.865957,0.869096,0.860906,0.867458,0.865305,0.002953,3
16,0.561365,0.044047,0.076006,0.005488,0.519485,0.613895,51,13,{'classifier__l2_regularization': 0.5194851192...,0.862973,0.867049,0.867868,0.857630,0.868550,0.864814,0.004081,4
5,1.178675,0.343545,0.172626,0.048729,0.737995,0.183492,4,85,{'classifier__l2_regularization': 0.7379954057...,0.862017,0.866776,0.865138,0.864046,0.864319,0.864459,0.001548,5


<p align="justify">
Hay que tener en cuenta que el ajuste está limitado por la cantidad de combinaciones diferentes de parámetros que se puntúan mediante la búsqueda aleatoria. De hecho, puede haber otros conjuntos de parámetros que conduzcan a resultados de generalización similares o mejores, pero que no se probaron en la búsqueda.
<br><br>
En la práctica, una búsqueda aleatoria de hiperparámetros suele ejecutarse con un gran número de iteraciones, lo cual es más costoso computacionalmente.

 # **<font color="DeepPink">Visualizacion</font>**

<p align="justify">
Como tenemos más de $2$ hiperparámetros en nuestra búsqueda aleatoria, no podemos visualizar los resultados usando un gráfico bidimencional como por ejemplo un mapa de calor. Sin embargo, es posible visualizar dicho resultado mediante el gráfico <code>parallel_coordinates</code> de Plotly. Este gráfico permite visualizar y comparar múltiples variables numéricas en un mismo gráfico utilizando coordenadas paralelas.

[Documentación](https://plotly.com/python/parallel-coordinates-plot/)

In [25]:
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 18 to 12
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   mean_fit_time                        20 non-null     float64
 1   std_fit_time                         20 non-null     float64
 2   mean_score_time                      20 non-null     float64
 3   std_score_time                       20 non-null     float64
 4   param_classifier__l2_regularization  20 non-null     object 
 5   param_classifier__learning_rate      20 non-null     object 
 6   param_classifier__max_leaf_nodes     20 non-null     object 
 7   param_classifier__min_samples_leaf   20 non-null     object 
 8   params                               20 non-null     object 
 9   split0_test_score                    20 non-null     float64
 10  split1_test_score                    20 non-null     float64
 11  split2_test_score                

In [26]:
cv_results = cv_results.drop(columns=['std_test_score','rank_test_score','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','mean_fit_time','std_fit_time','mean_score_time','std_score_time'])

In [27]:
cv_results = cv_results.rename(columns={'param_classifier__l2_regularization':'l2_regularization',
                                'param_classifier__learning_rate':'learning_rate',
                                'param_classifier__max_leaf_nodes':'max_leaf_nodes',
                                'param_classifier__min_samples_leaf':'min_samples_leaf',
                                'mean_test_score':'accuracy'})

In [28]:
cv_results = cv_results.astype("float")

In [29]:
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 18 to 12
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   l2_regularization  20 non-null     float64
 1   learning_rate      20 non-null     float64
 2   max_leaf_nodes     20 non-null     float64
 3   min_samples_leaf   20 non-null     float64
 4   accuracy           20 non-null     float64
dtypes: float64(5)
memory usage: 960.0 bytes


In [30]:
px.parallel_coordinates(cv_results,
                        color="accuracy",
                        color_continuous_scale=px.colors.sequential.Magma_r)

<p align="justify">
El gráfico de coordenadas paralelas mostrará los valores de los hiperparámetros en diferentes columnas, mientras que la métrica de rendimiento (accuracy) está codificada por colores. Por lo tanto, podemos inspeccionar rápidamente si hay un rango de hiperparámetros que funcionan o no.
<br><br>
Es posible seleccionar un rango de resultados haciendo clic y manteniendo presionado cualquier eje del gráfico de coordenadas paralelas. Luego puede deslizar (mover) la selección de rango y cruzar dos selecciones para ver las intersecciones. Podemos deshacer una selección haciendo clic una vez más en el mismo eje.
<br><br>
En particular para esta búsqueda de hiperparámetros, es interesante confirmar que las líneas negras (modelos de mejor rendimiento) alcanzan valores intermedios para la tasa de aprendizaje y regularización l2.
<br><br>
Los otros hiperparámetros tienen que ver con el tamaño de los áboles y la cantidad de datos en los nodos hoja. Estos tienen valores de medios a altos.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Generamos un Pipeline.
<br>  
✅ Separamos en un conjunto de entrenamiento y un conjunto de prueba.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo a través del randomized search.
<br>
✅ Visualizamos los valores de los hiperparámetros con un gráfico de coordenadas paralelas.


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
